In [ ]:
import matplotlib.pyplot as plt 

In [ ]:
def plot_single_experiemnt(agent_name, mid_price, bid_price, ask_price, rewards):

    # Plot spread
    plt.figure(figsize=(10,4), dpi=200)
    plt.plot(mid_price, label='mid price')
    plt.fill_between(range(len(mid_price)), bid_price, ask_price, alpha=0.5)
    plt.plot(bid_price, label='bid price')
    plt.plot(ask_price, label='ask price')
    plt.legend()
    plt.title(f'Spread Over Time ({agent_name})')
    plt.show()
    
    # Plot rewards
    plt.figure(figsize=(10,4), dpi=200)
    plt.plot(rewards)
    plt.title(f'Cumulative Reward Over Time ({agent_name})')
    plt.show()

In [ ]:
def single_experiment(agent_name, agent, env, show=True):

    # Record
    rewards = []
    mid_price = []
    bid_price = []
    ask_price = []
    volume = []
    cash = []

    # Initialize
    disc_reward = 0
    state = env.state()

    # Experiment
    while env.t < env.T:

        # Random agent
        if isinstance(agent, RandomAgent):
            action = agent.get_action()
            state, reward, done = env.step(np.array(action))

        # Q-learning agent
        elif isinstance(agent, QAgent):
            action = agent.get_action(state=state)
            next_state, reward, done = env.step(np.array(action))
            agent.update_Q(action, reward, state, next_state)
            agent.decay_epsilon()
            state = next_state

        # Multi-armed bandit agents
        else:
            action = agent.get_action()
            state, reward, done = env.step(np.array(action))
            agent.update_Q(action, reward)
            if isinstance(agent, DecayEpsilonGreedyAgent):
                agent.decay_epsilon()

    
        # Record
        disc_reward += reward  
        rewards.append(disc_reward)
        mid_price.append(env.mid)
        bid_price.append(env.mm_bid)
        ask_price.append(env.mm_ask)
        volume.append(state[0])
        cash.append(env.X_t)

    # Show single experiment results
    if show:
        env.render()
        plot_single_experiemnt(agent_name, mid_price, bid_price, ask_price, rewards)

    return rewards